<a href="https://colab.research.google.com/github/carolinampessoa/TechChallengeFase3/blob/main/FIAP_Tech_Challenge_Amazon_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive # type: ignore
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu114
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets
!pip install -U bitsandbytes

Looking in indexes: https://download.pytorch.org/whl/cu114
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.8 MB/s eta 0:00:00


In [3]:
!pip install "unsloth[torch,accelerate,trl]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.5/256.5 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.5/317.5 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [4]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
import pandas as pd
from trl import SFTTrainer
from transformers import TrainingArguments
import unicodedata
import os
import re

# Desativar o logging do wandb
os.environ["WANDB_DISABLED"] = "true"

TRAIN_DATA_PATH = "/content/drive/MyDrive/tc3/trn.json"
TEST_DATA_PATH = "/content/drive/MyDrive/tc3/tst.json"

OUTPUT_SANITIZED_TRAIN_DATA_PATH = "/content/drive/MyDrive/tc3/sanitized_trn.json"
OUTPUT_SANITIZED_TEST_DATA_PATH = "/content/drive/MyDrive/tc3/sanitized_tst.json"

OUTPUT_TRAIN_DATASET = "/content/drive/MyDrive/tc3/formatted_train_dataset.json"
OUTPUT_TEST_DATASET = "/content/drive/MyDrive/tc3/formatted_test_dataset.json"


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
def sanitize_dataset(input_file_path, output_file_path):
  # Carregar o arquivo original
  with open(input_file_path, "r") as file:
      data = [json.loads(line) for line in file]

  # Filtrar as colunas "title" e "content"
  filtered_data = [{"title": item["title"], "content": item["content"]} for item in data]

  # Contagem de vazios, nulos e duplicados
  empty_titles = 0
  null_titles = 0
  empty_contents = 0
  null_contents = 0
  duplicate_contents = 0

  # Rastrear duplicados na coluna "content"
  content_counts = {}

  # Contar os valores vazios, nulos e duplicados
  for item in data:
      # Contando Titles
      if item['title'] == "":
          empty_titles += 1
      elif item['title'] is None:
          null_titles += 1

      # Contando Contents
      if item['content'] == "":
          empty_contents += 1
      elif item['content'] is None:
          null_contents += 1

      # Contar duplicados na coluna "content"
      content = item['content']
      if content in content_counts:
          content_counts[content] += 1
      else:
          content_counts[content] = 1

  # Contar valores duplicados
  duplicate_contents = sum(1 for count in content_counts.values() if count > 1)

  total_titles = len(data)
  total_contents = len(data)

  # Exibir os resultados
  print(f"Total de Titles vazios: {empty_titles}")
  print(f"Total de Contents vazios: {empty_contents}")
  print(f"Total de Titles: {total_titles}")
  print(f"Total de Contents: {total_contents}")
  print(f"Total de Contents duplicados: {duplicate_contents}")

  # Retirar as colunas "title" e "content" vazias
  filtered_data = [item for item in data if item['title'] and item['content']]

  # Remover linhas com valores duplicados na coluna "content"
  seen_contents = set()
  duplicated_data = []
  for item in filtered_data:
      if item['content'] not in seen_contents:
          duplicated_data.append(item)
          seen_contents.add(item['content'])

  # Contar duplicados após a remoção
  duplicated_content_counts = {}
  for item in duplicated_data:
      content = item['content']
      if content in duplicated_content_counts:
          duplicated_content_counts[content] += 1
      else:
          duplicated_content_counts[content] = 1

  remaining_duplicates = sum(1 for count in duplicated_content_counts.values() if count > 1)

  # Contagem final
  empty_titles = 0
  null_titles = 0
  empty_contents = 0
  null_contents = 0

  for item in duplicated_data:
      # Contando Titles
      if item['title'] == "":
          empty_titles += 1
      elif item['title'] is None:
          null_titles += 1

      # Contando Contents
      if item['content'] == "":
          empty_contents += 1
      elif item['content'] is None:
          null_contents += 1

  total_titles = len(duplicated_data)
  total_contents = len(duplicated_data)

  print(f"------------------------Após a limpeza------------------------")
  # Exibir os resultados separados
  print(f"Total de Titles vazios: {empty_titles}")
  print(f"Total de Contents vazios: {empty_contents}")
  print(f"Total de Titles: {total_titles}")
  print(f"Total de Contents: {total_contents}")
  print(f"Total de Contents duplicados restantes: {remaining_duplicates}")

  # Salvar em um novo arquivo JSON
  with open(output_file_path, "w") as outfile:
      json.dump(duplicated_data, outfile, indent=4)

  print(f"Novo arquivo salvo em: {output_file_path}")

In [6]:
sanitize_dataset(TRAIN_DATA_PATH, OUTPUT_SANITIZED_TRAIN_DATA_PATH)
sanitize_dataset(TEST_DATA_PATH, OUTPUT_SANITIZED_TEST_DATA_PATH)

Total de Titles vazios: 126834
Total de Contents vazios: 749901
Total de Titles: 2248619
Total de Contents: 2248619
Total de Contents duplicados: 81091
------------------------Após a limpeza------------------------
Total de Titles vazios: 0
Total de Contents vazios: 0
Total de Titles: 1173617
Total de Contents: 1173617
Total de Contents duplicados restantes: 0
Novo arquivo salvo em: /content/drive/MyDrive/tc3/sanitized_trn.json
Total de Titles vazios: 55556
Total de Contents vazios: 323121
Total de Titles: 970237
Total de Contents: 970237
Total de Contents duplicados: 29423
------------------------Após a limpeza------------------------
Total de Titles vazios: 0
Total de Contents vazios: 0
Total de Titles: 531945
Total de Contents: 531945
Total de Contents duplicados restantes: 0
Novo arquivo salvo em: /content/drive/MyDrive/tc3/sanitized_tst.json


In [7]:
# Função para processar e formatar o dataset
def processar_formatar_dataset(input_file_path, output_file_path, sample_size=None):

    print(f"Carregando dataset de: {input_file_path}")

    # Carrega o arquivo JSON completo (array de objetos)
    with open(input_file_path, 'r', encoding='utf-8') as file:
        json_data = json.load(file)

    print(f"Total de registros carregados: {len(json_data)}")

    # Filtra apenas os registros que têm 'title' e 'content'
    data = []
    for item in json_data:
        if isinstance(item, dict) and 'title' in item and 'content' in item:
            title = item['title']
            content = item['content']
            data.append({'title': title, 'content': content})
        else:
            print(f"Registro inválido ignorado: {item}")

    print(f"Registros válidos encontrados: {len(data)}")

    # Converte a lista de dicionários em um DataFrame do pandas
    df = pd.DataFrame(data)

    # Função para normalizar o texto
    def normalize_text(text):
        # Check if text is a string before processing
        if not isinstance(text, str):
            return ""
        # Remove caracteres de controle
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        # Remove espaços extras
        text = re.sub(r'\s+', ' ', text).strip()
        return text


    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalize_text)
    df['content'] = df['content'].apply(normalize_text)

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Answer the question based on the product title."
    df['input'] = df['title'].apply(lambda x: f"What is '{x}'?")
    df['output'] = df['content']

    # Se 'sample_size' estiver definido, pega uma amostra do DataFrame
    if sample_size is not None:
        df = df.sample(n=sample_size, random_state=42).reset_index(drop=True)

    # Converte para uma lista de dicionários
    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records')

    # Salva o resultado em um arquivo JSON
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4)

    print(f"Dataset processado e salvo em '{output_file_path}'. Total de registros: {len(df)}")

In [ ]:
processar_formatar_dataset(OUTPUT_SANITIZED_TRAIN_DATA_PATH, OUTPUT_TRAIN_DATASET, sample_size = 500000)
processar_formatar_dataset(OUTPUT_SANITIZED_TEST_DATA_PATH, OUTPUT_TEST_DATASET, sample_size = 250000)

Carregando dataset de: /content/drive/MyDrive/tc3/sanitized_trn.json


In [ ]:
train_dataset = load_dataset("json", data_files=OUTPUT_TRAIN_DATASET, split="train")
test_dataset = load_dataset("json", data_files=OUTPUT_TEST_DATASET, split="train")

In [ ]:
# Define o template de prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:
# Função para formatar os prompts
def formatar_prompts(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Formata o texto usando o template
        text = alpaca_prompt.format(instruction, input_text, output)
        texts.append(text)
    return { "text": texts }

In [ ]:
# Aplica a formatação nos datasets de treinamento e teste
train_dataset = train_dataset.map(formatar_prompts, batched=True, remove_columns=train_dataset.column_names)
test_dataset = test_dataset.map(formatar_prompts, batched=True, remove_columns=test_dataset.column_names)

In [34]:
#!pip install unsloth-zoo # Install the missing dependency
#!pip install unsloth
#!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes triton

In [ ]:
# Configurações do modelo
max_seq_length = 256
dtype = torch.bfloat16
load_in_4bit = True

In [ ]:
# Carrega o modelo e o tokenizer usando o unsloth
model_name = "unsloth/llama-3-8b-bnb-4bit"  # Modelo quantizado em 4 bits
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

In [ ]:
# Configura o modelo para Fine-Tuning com LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_llama_8b_4bit_demo",
    num_train_epochs=1,          # Apenas 1 época para demonstração
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    eval_strategy="steps",
    save_strategy="steps",
    logging_steps=50,
    learning_rate=2e-4,
    fp16=True, # Changed from bf16=True to fp16=True
    optim="adamw_8bit",
    report_to="none",
)

In [ ]:
# Define o Trainer usando o SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # Added the evaluation dataset
    args=training_args,
    max_seq_length=max_seq_length,
    packing=False,
    dataset_text_field="text",
)

In [ ]:
trainer.train()

In [ ]:
# Prepara o modelo para inferência
FastLanguageModel.for_inference(model)

In [ ]:
# Salva o modelo treinado
model.save_pretrained("/content/drive/MyDrive/tc3/trained_model_llama_8b_4bit_demo")
tokenizer.save_pretrained("/content/drive/MyDrive/tc3/trained_model_llama_8b_4bit_demo")

print("Treinamento concluído e modelo salvo.")

In [28]:
#Teste do Modelo Treinado
# install the unsloth package
#!pip install unsloth
# Carrega o modelo treinado
from unsloth import FastLanguageModel
model_name = "/content/drive/MyDrive/tc3/trained_model_llama_8b_4bit_demo"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)

# Prepara um prompt de teste
prompt = alpaca_prompt.format(
    "Answer the question based on the product title.",
    "What is 'Girls Ballet Tutu Neon Pink'?",
    ""
)

# Tokeniza e gera a saída
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=50)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

==((====))==  Unsloth 2025.9.11: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Device does not support bfloat16. Will change to float16.


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the product title.

### Input:
What is 'Girls Ballet Tutu Neon Pink'?

### Response:
Girls Ballet Tutu Neon Pink. 100% polyester. 100% polyester. 100% polyester. 100% polyester. 100% poly
